In [2]:
%pip install -q imageio
%pip install -q opencv-python
%pip install -q matplotlib
%pip install -q git+https://github.com/tensorflow/docs

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Package 'tensorflow-docs' requires a different Python: 3.8.10 not in '>=3.9'
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from tensorflow_docs.vis import embed


2024-06-02 17:47:15.797547: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-02 17:47:15.798935: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-02 17:47:15.829935: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-02 17:47:15.830394: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-02 17:47:16.466755: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

ModuleNotFoundError: No module named 'tensorflow_hub'

In [47]:
#@title Helper functions for visualization

# Dictionary that maps from joint names to keypoint indices.
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# Maps bones to a matplotlib color name.
KEYPOINT_EDGE_INDS_TO_COLOR = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def label_keypoints(image_path,static_keypoints):
    image = cv2.imread(image_path)
    
    labeled_keypoints = {}
    for keypoint, index in KEYPOINT_DICT.items():
        y, x, conf = static_keypoints[0][0][index]
        labeled_keypoints[keypoint] = {
            'y': y,
            'x': x,
            'confidence': conf
        }
    
    return labeled_keypoints

In [38]:
module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
input_size = 192

def movenet(input_image):
  model = module.signatures['serving_default']

  # SavedModel format expects tensor type of int32.
  input_image = tf.cast(input_image, dtype=tf.int32)
  # Run model inference.
  outputs = model(input_image)
  # Output is a [1, 1, 17, 3] tensor.
  keypoints_with_scores = outputs['output_0'].numpy()
  return keypoints_with_scores

In [39]:
!curl -o input_image.jpeg https://images.pexels.com/photos/4384679/pexels-photo-4384679.jpeg --silent

In [40]:
# Load the input image.
image_path = 'input_image.jpeg'
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image)

In [51]:
# Resize and pad the image to keep the aspect ratio and fit the expected size.
input_image = tf.expand_dims(image, axis=0)
input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

# Run model inference.
keypoints_with_scores = movenet(input_image)

keypointLeft = 'left_hip'
keypointRight = 'right_hip'

labeled_keypoints = label_keypoints(image_path, keypoints_with_scores)
confidence_percent_left = labeled_keypoints[keypointLeft]["confidence"] * 100
confidence_percent_right = labeled_keypoints[keypointRight]["confidence"] * 100

print(f'Informações sobre {keypointLeft}:')
print(f'y={labeled_keypoints[keypointRight]["y"]:.2f}, x={labeled_keypoints[keypointLeft]["x"]:.2f}, confidence={confidence_percent_left:.2f}%')

print(f'\nInformações sobre {keypointRight}:')
print(f'y={labeled_keypoints[keypointRight]["y"]:.2f}, x={labeled_keypoints[keypointRight]["x"]:.2f}, confidence={confidence_percent_right:.2f}%')